In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gnss import codes
from gnss.receiver import channels
from gnss.signals import Signal
from gnss.receiver import sources
from gnss.filters import iir_filter
from gnss.acquisition import coarse, fine
from gnss.receiver import outputs

In [2]:
%reload_ext autoreload
%autoreload 2

In this notebook we will attempt to track a signal.

In [3]:
f_center = 1.25e6
f_samp = 5e6
filepath = '../../data/g072602f.dat'
source = sources.FileSignalSource(filepath, f_samp=f_samp, f_center=f_center)
source.load()
print('we have {0} seconds of data'.format(source.buffer_size / source.f_samp))

we have 1.048576 seconds of data


In order to track with a DLL, we need to perform early, prompt, and late correlation. We may also wish to add a search dimension over doppler frequency. Thus, we make the function `correlate`.

In [4]:
%%writefile ../../gnss/tracking/tracking.py

import numpy


def code_samples(signal, f_dopp, t, chip=0.):
    f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
    return 1. - 2. * signal.code.sequence[(numpy.floor(chip + t * f_chip) % len(signal.code.sequence)).astype(int)]


class Correlator:
    
    def __init__(self, chip_delays=[-.5, 0., .5], doppler_offsets=[]):
        self.chip_delays = chip_delays
        self.doppler_offsets = doppler_offsets
    
    def correlate(self, signal, source, block_size, time, chip, f_dopp, theta):
        samples, time = source.get(block_size, time)
        # TODO handle changes in time
        t = numpy.arange(block_size) / source.f_samp
        carrierless = samples * numpy.exp(-2j * numpy.pi * (source.f_center + f_dopp) * t - 1j * theta)
        codeless = samples * code_samples(signal, f_dopp, t, chip)
        chip_delay_outputs = (numpy.sum(carrierless * code_samples(signal, f_dopp, t, chip + delay)) \
                              for delay in self.chip_delays)
        doppler_offset_outputs = (np.sum(codeless \
                    * numpy.exp(-2j * numpy.pi * (source.f_center + f_dopp + dopp_offset) * t - 1j * theta)) \
                    for dopp_offset in self.doppler_offsets)
        return chip_delay_outputs, doppler_offset_outputs
    
    
def delay_discriminator(early, prompt, late, delay=.5):
    return delay * (numpy.abs(early) - numpy.abs(late)) / (numpy.abs(late) + numpy.abs(early) + 2 * numpy.abs(prompt))

def costas_discriminator(prompt):
    '''
    Return Costas discriminator output for phase.
    '''
    return numpy.real(prompt) * numpy.imag(prompt) / numpy.abs(prompt)**2

Overwriting ../../gnss/tracking/tracking.py


In [5]:
from gnss.tracking import tracking

In [19]:
svid = 21
signal = Signal.GPSL1CA(svid)

c_acq = coarse.CoarseAcquirer(source, 5e-3, 3)
f_acq = fine.FineAcquirer(source, 2e-3, 1e-3, 29)

In [20]:
c_acq.acquire(signal)
chip0 = c_acq.chip
f_dopp = c_acq.f_dopp
time = c_acq.time
snr = c_acq.snr
print(chip0, f_dopp, time, snr)

850.9314 2000.0 0.0 22.6375384847


In [21]:
f_acq.acquire(signal, time, chip0, f_dopp)
f_dopp = f_acq.f_dopp
time = f_acq.time
print(chip0, f_dopp, time)

850.9314 1952.93614722 0.0


In [22]:
step = 2.0006e-3
f_update = 1. / step
block_length = 2e-3
block_size = block_length * source.f_samp

dll_filter = iir_filter.FirstOrderLowpass(10, f_update)
# pll_filter = iir_filter.FirstOrderLowpass(12, f_update)
# pll_filter = iir_filter.SecondOrderLowpass(omega_n=5 * 12, zeta=1.7, fs=f_update)
pll_filter = iir_filter.SecondOrderLowpassV2(omega_n=5 * 12, zeta=1.7, fs=f_update)

delay = .5
correlator = tracking.Correlator(chip_delays=[-delay, 0., delay])

output_buff_size = 1000
tracking_outputs = {
    'phase': {'size': output_buff_size, 'dtype': np.float},
    'chip': {'size': output_buff_size, 'dtype': np.float},
    'i_corr': {'size': output_buff_size, 'dtype': np.float},
    'q_corr': {'size': output_buff_size, 'dtype': np.float}
    }
output = outputs.TrackingOutputBuffer(**tracking_outputs)

In [26]:
time = 0.0
output.clear()

In [27]:
theta = 0.
chip = chip_out = chip0
while time < source.max_time() - block_length:
    code_corr, dopp_corr = correlator.correlate(signal, source, block_size, time, chip, f_dopp, theta)
    early, prompt, late = code_corr
    chip_error = tracking.delay_discriminator(early, prompt, late)
    phase_error = -tracking.costas_discriminator(prompt)  # minus sign used b/c phase is subtracted in signal correlator
    chip_error = dll_filter.f(chip_error)
    phase_error = pll_filter.f(phase_error)
    # error correction
    chip -= chip_error
    theta -= phase_error
    # carrier-aiding and propagation
    time += step
    f_chip = signal.code.rate * (1. + f_dopp / signal.f_carrier)
    chip += step * f_chip
    theta += step * (source.f_center + f_dopp) * 2. * np.pi  # TODO: why do both theta update lines work?
    # outputs
    chip_out = chip_out - step * f_chip
    output.push(**{'phase': phase_error, 'chip': chip_error, 'i_corr': np.real(prompt), 'q_corr': np.imag(prompt)})

In [28]:
output.plot()
plt.show()

.

.

.

Things:

- adding time to `t` array in correlator works, but I don't think it's a good tracking method
- updating using `phase += step * 2 * pi * (f_center + f_dopp)` works
    - need to remember to subtract `theta` inside correlator
    - because theta is used to subtract off, the resulting error will be negative
        - should we change the costas correlator output or "add phase error" to phase?
- `theta` update also works with `phase += step * 2 * pi * f_dopp`--this seems strange
    - problem understood: verified using `step = 2.0006e-3`
        - `f_center` rate is `8e-7`

.

.

.

When tracking, we assume that we have already aquired the doppler and code phase of the signal.

In tracking, we have a reference signal that we use to wipe off the code and carrier from the signal. We generate it using our estimates for code chip phase, doppler frequency, and signal phase--$(\eta, f_d, \theta)$.

If we have the 3 correlator outputs, what error should we provide as feedback to our DLL loop filter?

Let $E$, $P$, and $L$ be the early, prompt, and late correlation outputs respectively.
Each has a real and imaginary component--or a magnitude and a phase.

We can always fit a polynomial of degree 2 through the magnitudes of the three points--this should yield the ML estimate for the correct code phase. Then this estimate minus the prompt phase yields the error. 

The correlator outputs are:

$$
x(t) = AN(t)C(t)\cos((\omega_{L1} + \omega_d)t + \theta) \\
x(n) = AN(t)C(t)\cos((\omega_{L1} + \omega_d)t + \theta) \\
Z_p = 
$$

Consider the points $(-d, y_e), (0, y_p), (d, y_l)$.

We can solve $\mathbf{y} = X\mathbf{a}$ for our coefficient vector $\mathbf{a}$.

$$
\newcommand{\va}{\mathbf{a}}
\newcommand{\vy}{\mathbf{y}}
\begin{align*}
\vy &= X \va \\
\va &= X^{-1} \vy \\
X &= \left(\begin{array}{ccc}
1 & -d & d^2 \\
1 & 0 & 0 \\
1 & d & d^2 \\
\end{array}\right) \\
\end{align*}
$$

$$
\begin{align*}
|X| &= 2d^3 \\
adj(X) &= \left(\begin{array}{ccc}
0 & -d^2 & d \\
2d^3 & 0 & 2d \\
0 & d^2 & d \\
\end{array}\right)^T \\
\end{align*}
$$

$$
\begin{align*}
X^{-1} &= \frac{1}{2d^3} \left(\begin{array}{ccc}
0 & 2d^3 & 0 \\
-d^2 & 0 & d^2 \\
d & 2d & d \\
\end{array}\right) \\
\Rightarrow \\
\va = \left[ y_p, \frac{1}{2d}(y_l - y_e), \frac{1}{2d^2}(y_e + y_l + 2y_p) \right]
\end{align*} \\
$$

The next question is, where does the maximum occur for this function. It occurs where $\frac{d}{dx}\left(a_0 + a_1 x + a_2 x^2\right) = 0 \Rightarrow a_1 + 2a_2x = 0 \Rightarrow x = -\frac{a_1}{2a_2}$. The error is $-x$, so our feedback error has the full form:

$$
e = \frac{a_1}{2a_2} = \frac{d(y_l - y_e)}{y_e + y_l + 2y_p}
$$

.

.

We need to add in a loop filter so we're not thrown off by spurious noise.

We need to design some filter loop parameters.

If $L(n)$ is our raw discriminator output, we generate a filtered output $L_p(n) = L_p(n-1) + 4TB(L(n) - L_p(n-1))$ where $T$ is the integration time and $B$ is the noise equivalent bandwidth. Note that with a longer integration time, a difference in current and past output is weighted more heavily.

We can rewrite this equation as:

$L_p(n) = (1 - 4TB)L_p(n-1) + 4TBL(n)$

Next, we want a feedback error for our PLL loop filter.

The error between the current doppler frequency estimate and actual estimate will be the rate of change of the phase. We can just use the current output and last output of our phase discriminator, which we describe next.

We will need a phase discriminator to tell us the difference between the phase f

$$
\newcommand{\re}{\text{Re}}
\newcommand{\im}{\text{Im}}
L_\theta = \re{Z_P}\im{Z_P}
 = \cos{\Delta\theta}\sin{\Delta\theta}
$$
`e_pll = np.real(prompt) * np.imag(prompt) / np.abs(prompt)`

Our received signal after ADC is:

$$ s(n) = \sqrt(P_c) D(n) C(n) \cos((\omega_{IF} + \omega_d)\frac{n}{fs} + \theta) + \epsilon(n)$$

At baseband, we have:

$$ s(n) = \sqrt(P_c) D(n) C(n) \exp(j(\omega_d\frac{n}{fs} + \theta)) + \epsilon_A(n)\exp(j\epsilon_\theta(n)) $$

After wiping off estimated doppler, we have:

$$ s(n) = \sqrt(P_c) D(n) C(n) \exp(j(2\pi\Delta f_d\frac{n}{fs} + \theta)) + \epsilon_A(n)\exp(j\epsilon_\theta(n)) $$

Let's briefly ignore the navigation data $D$, code $C$, and amplitude terms:

$$ s(n) = \exp(j(2\pi\Delta f_d\frac{n}{fs} + \theta)) + N_0'(n) $$

If we sum this over our integration time $T$, which corresponds to a block length of $N$ samples, we get:

$$ \begin{align*}
\sum_{n=0}^{N-1} s(n) &= \exp(j\theta)\sum_{n=0}^{N-1} \left[\exp(j2\pi\Delta f_d\frac{n}{fs}) + \epsilon_A(n)\exp(j\epsilon_\theta(n))\right] \\
&= \exp(j\theta)\frac{1 - \exp(j2\pi\Delta f_d\frac{N}{fs})}{1 - \exp(j2\pi\Delta f_d\frac{1}{fs})}+ \sum_{n=0}^{N-1} \epsilon_A(n)\exp(j\epsilon_\theta(n))\\
&= \exp(j\theta)\frac{\exp(j\pi\Delta f_d\frac{N}{fs})}{\exp(j\pi\Delta f_d\frac{1}{fs})}\frac{\exp(-j\pi\Delta f_d\frac{N}{fs}) - \exp(j\pi\Delta f_d\frac{N}{fs})}{\exp(-j\pi\Delta f_d\frac{1}{fs}) - \exp(j\pi\Delta f_d\frac{1}{fs})}+ \epsilon_A'\exp(j\epsilon_\theta')\\
&=\exp(j\theta)\exp\left(j\pi\Delta f_d\frac{N-1}{fs}\right)\frac{\sin\pi\Delta f_d T}{\sin\pi\frac{\Delta f_d}{fs}}+ \epsilon_A'\exp(j\epsilon_\theta')\\
\end{align*}$$

Since $\Delta f_d << fs \Rightarrow \sin\pi\frac{\Delta f_d}{fs} \approx \pi\frac{\Delta f_d}{fs}$ and since $N/T = 1/fs$, we can write:

$$ \begin{align*}
S_N = \sum_{n=0}^{N-1} s(n) &=\exp(j\theta)\exp\left(j\pi\Delta f_d\frac{N-1}{fs}\right)N\frac{\sin\pi\Delta f_d T}{\pi\Delta f_d T}+ \epsilon_A'\exp(j\epsilon_\theta')\\
&= N\exp(j\theta)\exp\left(j\pi\Delta f_d\frac{N-1}{fs}\right)\text{sinc}\pi\Delta f_d T + \epsilon_A'\exp(j\epsilon_\theta')\\
&\approx N\exp(j\theta)\exp\left(j\pi\Delta f_d\frac{N-1}{fs}\right) + \epsilon_A'\exp(j\epsilon_\theta')\\
\end{align*}$$

Our noise random variable for the integrated signal is--in magnitude phase notation--$\epsilon_A'\exp(j\epsilon_\theta')$. The magnitude $\epsilon_A'$ has variance $\sigma^2 = N\sigma_{\epsilon_A}^2$, and the phase is uniformly distributed between $(0, 2\pi)$.

